Calculate functionals with symbolics 
create kernel 

Create points 

construnct distance tnsors 


apply kernel to a distance tensor 

assemble RHS and matrices 

Assemble DAE 

Solve with diffeq.jl or custom code 





In [1]:
using Revise
using CairoMakie
using Symbolics
using Latexify
using DifferentialEquations
using Sundials
cd("..")
include("RBFunctions.jl")
cd("circular_stokes_flow")

In [2]:
@variables ϵ r x₁ x₂ t;
const nu = 1.0
#ϕ = 1//945 * ((ϵ*r)^5 +15*(ϵ*r)^3 + 105*(ϵ*r)^2 + 945*(ϵ*r)+ 945)* exp(-ϵ*r)
ϕ = exp(-r^2*ϵ^2)
ϕ = substitute(ϕ, r=>sqrt(x₁^2 + x₂^2)) 
#display(ϕ)  
Δ(exprs) = expand_derivatives((Differential(x₁)^2)(exprs) + (Differential(x₂)^2)(exprs))
∂₁(exprs) = expand_derivatives(Differential(x₁)(exprs))
∂₂(exprs) = expand_derivatives(Differential(x₂)(exprs))
∂ₜ(exprs) = expand_derivatives(Differential(t)(exprs))

Φ_div = ([-∂₂(∂₂(ϕ)) ∂₁(∂₂(ϕ)) 0.0 ; ∂₁(∂₂(ϕ)) -∂₁(∂₁(ϕ)) 0.0; 0.0 0.0 ϕ])
ΔΦ_div= Δ.([-∂₂(∂₂(ϕ)) ∂₁(∂₂(ϕ)); ∂₁(∂₂(ϕ)) -∂₁(∂₁(ϕ))])
Φ_curl = ([-∂₁(∂₁(ϕ)) -∂₁(∂₂(ϕ)); -∂₁(∂₂(ϕ)) -∂₂(∂₂(ϕ))])
Φ = [ϕ 0.0 0.0; 0.0 ϕ 0.0;0.0 0.0 ϕ]
#ΔΦ = [Δ(ϕ) 0 ; 0 Δ(ϕ)]
true_u₁ = -x₂*π*sin(π*0.5*(x₁*x₁ + x₂*x₂))*sin(π*t)
true_u₂ =  x₁*π*sin(π*0.5*(x₁*x₁ + x₂*x₂))*sin(π*t)
true_∂ₜu₁ = ∂ₜ(true_u₁)
true_∂ₜu₂  = ∂ₜ(true_u₂)
true_p = sin(x₁-x₂+t)
true_∂₁p = ∂₁(true_p)
true_∂₂p = ∂₂(true_p)
f₁ = ∂ₜ(true_u₁) - nu*Δ(true_u₁) + true_∂₁p
f₂ = ∂ₜ(true_u₂) - nu*Δ(true_u₂) + true_∂₂p


true_u₁ = eval(build_function(true_u₁,x₁, x₂, t))
true_u₂ = eval(build_function(true_u₂,x₁, x₂, t))
true_p = eval(build_function(true_p,x₁, x₂, t))
true_∂₁p = eval(build_function(true_∂₁p,x₁, x₂, t))
true_∂₂p = eval(build_function(true_∂₂p,x₁, x₂, t))
f₁ = eval(build_function(f₁,x₁, x₂, t))
f₂ = eval(build_function(f₂,x₁, x₂, t))
∂ₜu₁ = eval(build_function(true_∂ₜu₁,x₁, x₂, t))
∂ₜu₂  = eval(build_function(true_∂ₜu₂,x₁, x₂, t))
zero_func(x₁,x₂,t) = 0.0
#display(Φ)
#display(Φ[1,:])
λ₁(x) = x[1]
λ₂(x) = x[2]
λ₃(x) = x[3]
λ₄(x) = x[1]
λ₅(x) = x[2]

λ∂ₜu₁(x) = Δ(x[1]) - ∂₁(x[3]) 
λ∂ₜu₂(x) = Δ(x[2]) - ∂₂(x[3]) 
λ∇u(x) = ∂₁(x[1]) + ∂₂(x[2]) 


function construct_kernel_array(matrix_kernel,functionals1,functionals2)
    N1 = length(functionals1)
    N2 = length(functionals2)
    M = Matrix{typeof(matrix_kernel[1,1])}(undef,N1,N2)
    for j = 1:N2
        λⱼ = functionals2[j]
        v = [λⱼ(matrix_kernel[1,:]),λⱼ(matrix_kernel[2,:]),λⱼ(matrix_kernel[3,:])]
        for i = 1:N1
            λᵢ = functionals1[i]
            M[i,j] = λᵢ(v)
        end
    end
    return M 
end

function compile_kernel_array(M)
    N1 = size(M)[1]
    N2 = size(M)[2]
    P = Matrix{Function}(undef,N1,N2)
    for i = 1:N1
        for j = 1:N2
            #display(M[i,j])
            P[i,j] = eval(build_function(M[i,j], [x₁, x₂], ϵ))
        end
    end
    return P
end

function crete_block_point_tensors(p_list1,p_list2)
    N1 = length(p_list1)
    N2 = length(p_list2)
    M = Matrix{Array{Float64, 3}}(undef,N1,N2)
    for i in 1:N1
        for j in 1:N2
            M[i,j] = point_difference_tensor(p_list1[i],p_list2[j])
        end
    end

    return M
end
#point_difference_tensor(Internal_points,Internal_points)
function generate_block_matrices(function_array,tensor_array,param)
    n1,n2 = size(tensor_array)
    if size(tensor_array) != size(function_array)
        return ArgumentError("function array and tensor array size mismatch")
    end
    M = Matrix{Matrix}(undef,n1,n2)
    for i in 1:n1
        for j in 1:n2
            #display(function_array[i,j])
            #display(tensor_array[i,j])
            M[i,j] = apply(function_array[i,j], tensor_array[i,j], param)
        end
    end
    return M
end
function flatten(block_matrix)
    # flattens block matrices into usual matrices 
    n1,n2 = size(block_matrix)
    res = hcat(block_matrix[1,:]...)
    #println(size(res))
    for i in 2:n1
        row  = hcat(block_matrix[i,:]...)
        res = vcat(res,row)
        #println(size(row))
    end
    return res
end


flatten (generic function with 1 method)

In [3]:
#generate points for circular flow benchmark 
using DistMesh2D
fdistance(p) = sqrt(sum(p .^ 2)) - 1
fedgelength(p) = 1/(sqrt(sum(p .^ 2))+1)
boundingbox = [-1.1 -1.1; 1.1 1.1]
initdistance = 0.2
x, y = distmesh2d(fdistance, fedgelength, boundingbox, initdistance)

# generate points, reference solution, forcing vector, and boundary conditions 
tx = unique(filter(e -> !isequal(e,NaN) ,x))
ty = unique(filter(e -> !isequal(e,NaN) ,y))

points = hcat(tx,ty)'

Internal_points = []
Boundary_points = []
suma = 0
for i in 1:length(tx)
    if abs(sum(points[:,i] .^2) - 1) < 1e-8
        #println(i)
        suma +=1
        append!(Boundary_points,[points[:,i]])
    else
        append!(Internal_points,[points[:,i]])
    end
    
end

Internal_points = hcat(Internal_points...)
Boundary_points = hcat(Boundary_points...)
All_points = hcat(Internal_points,Boundary_points)
N_i = size(Internal_points)[2]
N_b = size(Boundary_points)[2]
N = N_i + N_b
println("total number of nodes: ",N)
println("number og internal_nodes: ",N_i)

total number of nodes: 90
number og internal_nodes: 57


In [109]:
Interpolating_kernel_array = construct_kernel_array(Φ_div,[λ₁,λ₂,λ₃,λ₄,λ₅],[λ₁,λ₂,λ₃,λ₄,λ₅])
ODE_kernel_array = construct_kernel_array(Φ_div,[λ∂ₜu₁,λ∂ₜu₂,λ∇u],[λ₁,λ₂,λ₃,λ₄,λ₅])
display(ODE_kernel_array)
display(Interpolating_kernel_array)
ODE_function_array = compile_kernel_array(ODE_kernel_array)
Interpolating_function_array = compile_kernel_array(Interpolating_kernel_array)
K_ = crete_block_point_tensors([Internal_points,Internal_points,Internal_points,Boundary_points,Boundary_points],
[Internal_points,Internal_points,Internal_points,Boundary_points,Boundary_points])
S_= crete_block_point_tensors([Internal_points,Internal_points,Internal_points],
[Internal_points,Internal_points,Internal_points,Boundary_points,Boundary_points])

K = flatten(generate_block_matrices(Interpolating_function_array,K_,3))
S = flatten(generate_block_matrices(ODE_function_array,S_,3))

C = S*inv(K)
A = C[:,1:3*N_i]
B = C[:,3*N_i+1:end]
E = zeros((3*N_i,3*N_i))
E[1:2*N_i,1:2*N_i] = I(2*N_i)
display(S)
#display(B)
#display(E)
println(cond(K))
println(cond(A-0.01*E))

println("max eigenval: ",maximum(real.(eigvals(A))))
"""assemble RHS vectors"""
f = generate_vector_function([f₁,f₂,zero_func],Internal_points)
f2 = generate_vector_function(f₁,f₂,Internal_points,Mixed=false)
g= generate_vector_function(true_u₁,true_u₂,Boundary_points;Mixed=false)
u_true = generate_vector_function(true_u₁,true_u₂,Internal_points;Mixed=false)
u_true = generate_vector_function([true_u₁,true_u₂],Internal_points)
p_true = generate_vector_function(true_p,Internal_points)
sol_true = generate_vector_function([true_u₁,true_u₂,true_p],Internal_points)
#display(A[2*N_i+1:end,2*N_i+1:end])
#minimum(abs.(inv(A)))

3×5 Matrix{Num}:
     (8//1)*(x₁^2)*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) + (56//1)*(x₂^2)*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16//1)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16//1)*(x₂^4)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16//1)*(x₁^2)*(x₂^2)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))  …  (16//1)*x₁*(x₂^3)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) + (16//1)*x₂*(x₁^3)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (48//1)*x₁*x₂*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))
 (16//1)*x₁*(x₂^3)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) + (16//1)*x₂*(x₁^3)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (48//1)*x₁*x₂*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))                                                                                                                         (8//1)*(x₂^2)*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) + (56//1)*(x₁^2)*(ϵ^6)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16//1)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16//1)*(x₁^4)*(ϵ^8)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (16

5×5 Matrix{Num}:
       (2//1)*(ϵ^2)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (4//1)*(x₂^2)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))  …  (4//1)*x₁*x₂*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))
 (4//1)*x₁*x₂*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))                                                                   (2//1)*(ϵ^2)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (4//1)*(x₁^2)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))
                   0.0                                                                                                              0.0
       (2//1)*(ϵ^2)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (4//1)*(x₂^2)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))     (4//1)*x₁*x₂*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))
 (4//1)*x₁*x₂*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))                                                                   (2//1)*(ϵ^2)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2)) - (4//1)*(x₁^2)*(ϵ^4)*exp(-(ϵ^2)*(sqrt(x₁^2 + x₂^2)^2))

171×237 Matrix{Float64}:
 -1296.0           113.839       -60.1067    …  -0.0273992    -8.48489e-5
   113.839       -1296.0          12.1072       -0.000210781  -1.72042e-5
   -60.1067         12.1072    -1296.0          -0.00660302   -0.00109453
  -205.247        -731.154      -143.447        -0.00137824   -6.42204e-5
    35.2646       -698.134       421.235        -1.28775e-5   -1.53723e-6
     0.0883862      11.6662       19.4137    …  -4.27863e-8   -2.67185e-8
    29.1802       -147.742      -639.508        -0.000288867  -6.81193e-5
     7.53424        45.8605     -110.608        -8.72261e-5   -3.80654e-5
     0.627512       32.2442       45.8605       -3.12896e-7   -1.12877e-7
     1.16327       -77.9929      193.71         -1.18824e-6   -2.40698e-7
     0.0124494       0.627512      7.53424   …  -5.84692e-8   -7.23024e-8
    -8.88268e-5     -0.070884     -0.278928     -2.23364e-7   -8.43377e-7
    -0.00225721     -0.231002     -2.14879      -4.39677e-5   -0.000173055
     ⋮      

3.1156811628828454e6
Inf
max eigenval: 0.0


(::var"#aa#6"{Vector{Function}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Int64, Int64}) (generic function with 1 method)

In [103]:

u0 = vcat(u_true(0),p_true(0))
display(u0)
t_interval = (0.0,2)
function stokes(du, u, p, t)
    A,B,g,f = p
    du .= A * u .+ B * g(t) .+ f(t)
    nothing
end
func = ODEFunction(stokes, mass_matrix = E)
prob = ODEProblem(func, u0, t_interval, [A,B,g,f])
sol = solve(prob, ABDF2())


171-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.3983963846461466
  0.16035678371952294
 -0.33393990265035167
 -0.5732978124894794
 -0.35390759448667325
 -0.08694637638377417
 -0.7588905529450286
 -0.9086037000970949
 -0.587316026587866
 -0.7415557877279925
 -0.8287358709046245
 -0.8978851034647041

retcode: Success
Interpolation: 3rd order Hermite
t: 69-element Vector{Float64}:
 0.0
 1.0e-6
 2.0e-6
 5.1678342030133615e-6
 3.684617623314698e-5
 0.00035362959653448316
 0.0019588988546038496
 0.003195858870840257
 0.0044328188870766646
 0.0060690953562660445
 0.008078007162628294
 0.010232363034191214
 0.012614922647156024
 ⋮
 1.6442695828605811
 1.6828764017922941
 1.7214832207240072
 1.7726371161822032
 1.8140839258021082
 1.8555307354220132
 1.8900592040519297
 1.9245876726818463
 1.951477606462958
 1.9783675402440697
 1.994963686241271
 2.0
u: 69-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  -0.33393990265035167, -0.5732978124894794, -0.35390759448667325, -0.08694637638377417, -0.7588905529450286, -0.9086037000970949, -0.587316026587866, -0.7415557877279925, -0.8287358709046245, -0.8978851034647041]
 [5.173632553215601e-6, 6.851892375493079e-6, 3.546282187203519e-6, 6.543115588073747e-6, 7.426270033851382e-6, 6.596725949436626e-6, 3.9996

In [104]:
function max_error(computed_sol::Function,reference_sol::Function,n::Int)
    range = LinRange(0.0,1.2,1000)
    m_array = zeros(length(range))
    for (i,t) in enumerate(range)
        m_array[i] = maximum(abs.(computed_sol(t)[1:n] - reference_sol(t)[1:n]))
    end
    return maximum(m_array)
end 


#maximum(vcat(sol_true.(range)...) .- vcat(sol.(range)...))
#maximum(hcat(u_true.(range)...)[1:2*N_i,:])- ((hcat(sol.(range)...)[1:2*N_i,:]))

#maximum(abs.(vcat(sol_true.(0:0.01:0.5)...) - vcat(sol(0:0.01:0.5)...)))
#println(maximum(vcat(sol.(0.0:0.01:2.0)...) .- vcat(u_true.(0.0:0.01:2.0)...)))
#tt = 0.2
#println(maximum(sol(tt)[1:2*N_i] .- u_true(tt)))
#display(sum(sol_true(1.2)[1:2*N_i,:] - u_true(1.2)))
#display(u_true(1.2))
#display(hcat(u_true.(range)...))
#display(hcat(sol_true.(range)...)[1:2*N_i,:])
#display(hcat(sol(range)...))
#sol.(range)
#f3 = generate_vector_function([f₁,f₂,zero_func],Internal_points)
#f4 = generate_vector_function(f₁,f₂,Internal_points,Mixed=false)
#display(f3(1.2))
#display(f4(1.2))
max_error(sol,sol_true,2*N_i)

0.0832173088093211

In [7]:
"""
# test the code on interpolation
M = construct_kernel_array(Φ_div,[λ₁,λ₂,λ₃],[λ₁,λ₂,λ₃])
O = compile_kernel_array(M)
int_points = hcat(generate_2D_equally_spaced_points(10)...)
tst_points, _ =  generate_2D_equally_spaced_points(20)
u₁_int = generate_vector_function(true_u₁,int_points)(1.3)
u₂_int = generate_vector_function(true_u₂,int_points)(1.3)
p_int = generate_vector_function(true_p,int_points)(1.3)

u₁_tst = generate_vector_function(true_u₁,tst_points)(1.3)
u₂_tst = generate_vector_function(true_u₂,tst_points)(1.3)
p_tst = generate_vector_function(true_p,tst_points)(1.3)


A_int = crete_block_point_tensors([int_points,int_points,int_points],[int_points,int_points,int_points])
A_tst = crete_block_point_tensors([tst_points,tst_points,tst_points],[int_points,int_points,int_points])
P_int = generate_block_matrices(O,A_int,2)
P_tst = generate_block_matrices(O,A_tst,2)
FF_int = flatten(P_int)
FF_tst = flatten(P_tst)

println("cond_number:", cond(FF_int))

reconstructed = FF_tst* (FF_int\ vcat(u₁_int,u₂_int,p_int))
error = reconstructed - vcat(u₁_tst,u₂_tst,p_tst)
fig, ax, hm = heatmap(tst_points[1,:],tst_points[2,:],error[324*2+1:324*3],aspect_ratio=:equal,colorbar=true)
Colorbar(fig[:, end+1], hm)
fig
"""

LoadError: syntax: invalid escape sequence